## 0. Docker
Это средство контейниризации. Вся суть контениризвации свести к минимуму, установку и настройку зависимостией, доп библиотек, дополнительного ПО, для запуска программы. Так же допиться переносимости и воспроизводимости результатов работы программы.

## 0.1 Image и контейнер
![](_static/image_vs_container.jpg)

In [6]:
!docker image ls

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE


In [8]:
!docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

Digest: sha256:f5233545e43561214ca4891fd1157e1c3c563316ed8e237750d59bde73361e77
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https

In [11]:
!docker image ls

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
hello-world         latest              e38bc07ac18e        3 days ago          1.85kB


Теперь посмотрим запущеные контейнеры

In [12]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [13]:
!docker container ls --all

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                      PORTS               NAMES
152153344bc8        hello-world         "/hello"            50 seconds ago      Exited (0) 49 seconds ago                       festive_goodall


### 0.2 Создание docker image
Image билдится (build) из файла специального формата который называется Dockerfile. Формат такой

```
FROM base_image
COPY something somewere
RUN some_command
```

создадим имедж с убутной и поставим туда python

In [42]:
!cd ./docker/img-ubuntu-python/ && docker build . -t img-ubuntu-python

Sending build context to Docker daemon   2.56kB
Step 1/2 : FROM ubuntu:16.04
 ---> c9d990395902
Step 2/2 : RUN apt-get update && apt-get install -y python3
 ---> Running in b741fc8dd8fd
Get:1 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [102 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [102 kB]
Get:4 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [78.5 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [102 kB]
Get:6 http://archive.ubuntu.com/ubuntu xenial/universe Sources [9802 kB]
Get:7 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [602 kB]
Get:8 http://security.ubuntu.com/ubuntu xenial-security/restricted amd64 Packages [12.7 kB]
Get:9 http://security.ubuntu.com/ubuntu xenial-security/universe amd64 Packages [431 kB]
Get:10 http://security.ubuntu.com/ubuntu xenial-security/multiverse amd64 Packages [3489 B]
Get:11 http:

Selecting previously unselected package file.
Preparing to unpack .../file_1%3a5.25-2ubuntu1_amd64.deb ...
Unpacking file (1:5.25-2ubuntu1) ...
Processing triggers for libc-bin (2.23-0ubuntu10) ...
Setting up mime-support (3.59ubuntu1) ...
Setting up libmpdec2:amd64 (2.4.2-1) ...
Setting up libsqlite3-0:amd64 (3.11.0-1ubuntu1) ...
Setting up libpython3.5-stdlib:amd64 (3.5.2-2ubuntu0~16.04.4) ...
Setting up python3.5 (3.5.2-2ubuntu0~16.04.4) ...
Setting up libpython3-stdlib:amd64 (3.5.1-3) ...
Setting up libmagic1:amd64 (1:5.25-2ubuntu1) ...
Setting up file (1:5.25-2ubuntu1) ...
Setting up python3 (3.5.1-3) ...
running python rtupdate hooks for python3.5...
running python post-rtupdate hooks for python3.5...
Setting up dh-python (2.20151103ubuntu1.1) ...
Processing triggers for libc-bin (2.23-0ubuntu10) ...
Removing intermediate container b741fc8dd8fd
 ---> c03900fbba20
Successfully built c03900fbba20
Successfully tagged img-ubuntu-python:latest


### 0.3 Docker run и exec
Для того чтобы создать из имеджа контейнер и запустить там что-нибудь используется коммада docker run

In [26]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [43]:
!docker run img-ubuntu-python cat /etc/lsb-release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=16.04
DISTRIB_CODENAME=xenial
DISTRIB_DESCRIPTION="Ubuntu 16.04.4 LTS"


Можно убедиться что мы на другой операционной системе

In [44]:
!cat /etc/lsb-release

DISTRIB_ID=LinuxMint
DISTRIB_RELEASE=18
DISTRIB_CODENAME=sarah
DISTRIB_DESCRIPTION="Linux Mint 18 Sarah"


In [9]:
!docker run --name ubuntu-python -d img-ubuntu-python bash -c 'while true; do sleep 60; done;' 

8d35d81ec7e522d60f2b555b803b28bfb1cc7fbc94fdc90912f092b9de061e81


In [10]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS               NAMES
8d35d81ec7e5        img-ubuntu-python   "bash -c 'while true…"   3 seconds ago       Up 1 second                             ubuntu-python
bea21b56943f        img-ubuntu-python   "bash -c 'while true…"   54 seconds ago      Up 53 seconds                           kind_babbage


In [12]:
!docker exec ubuntu-python ps ax 

  PID TTY      STAT   TIME COMMAND
    1 ?        Ss     0:00 bash -c while true; do sleep 60; done;
    7 ?        S      0:00 sleep 60
   14 ?        Rs     0:00 ps ax


Про интерактивный режим

## 0.4 Создание контейнера с ssh сервером

In [13]:
!cd ./docker/img-ubuntu-sshd/ && docker build . -t img-ubuntu-sshd

Sending build context to Docker daemon   2.56kB
Step 1/10 : FROM ubuntu:16.04
 ---> c9d990395902
Step 2/10 : RUN apt-get update && apt-get install -y openssh-server
 ---> Using cache
 ---> b1cd4579213a
Step 3/10 : RUN mkdir /var/run/sshd
 ---> Running in e48fc3612c4f
Removing intermediate container e48fc3612c4f
 ---> b4bfb3d20018
Step 4/10 : RUN echo 'root:pwd' | chpasswd
 ---> Running in acbd4d3219cf
Removing intermediate container acbd4d3219cf
 ---> f9de9f4c37b5
Step 5/10 : RUN sed -i 's/PermitRootLogin prohibit-password/PermitRootLogin yes/' /etc/ssh/sshd_config
 ---> Running in 48ae498c9cab
Removing intermediate container 48ae498c9cab
 ---> 5079295f9f12
Step 6/10 : RUN sed 's@session\s*required\s*pam_loginuid.so@session optional pam_loginuid.so@g' -i /etc/pam.d/sshd
 ---> Running in 19c904d684b6
Removing intermediate container 19c904d684b6
 ---> 2f0ca8f3e760
Step 7/10 : ENV NOTVISIBLE "in users profile"
 ---> Running in ade6149bba8f
Removing intermediate container ade6149bba8f
 ---

In [23]:
!docker run -d -p 22022:22 --name cont-ubuntu-sshd img-ubuntu-sshd 

docker: Error response from daemon: Conflict. The container name "/cont-ubuntu-sshd" is already in use by container "897e34c1bfba32e76c87c5e3ef60c187df9805b386f536e4526804081cd31204". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


### 0.4 Очистка

In [48]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND               CREATED              STATUS              PORTS                   NAMES
de88e7fea95d        img-ubuntu-sshd     "/usr/sbin/sshd -D"   About a minute ago   Up About a minute   0.0.0.0:22022->22/tcp   cont-ubuntu-sshd


In [49]:
!docker stop $(docker ps -q) # kill

de88e7fea95d


In [50]:
!docker container ls

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [51]:
!docker container ls --all 

CONTAINER ID        IMAGE               COMMAND               CREATED              STATUS                    PORTS               NAMES
de88e7fea95d        img-ubuntu-sshd     "/usr/sbin/sshd -D"   About a minute ago   Exited (0) 1 second ago                       cont-ubuntu-sshd


In [52]:
!docker container prune -f

Deleted Containers:
de88e7fea95dfaeb6da7086c36300588015ddf92cef85c1be38673af1d32c9d3

Total reclaimed space: 32.32kB


In [53]:
!docker system prune 

WARNING! This will remove:
        - all stopped containers
        - all networks not used by at least one container
        - all dangling images
        - all build cache
Are you sure you want to continue? [y/N] ^C


In [55]:
!docker system prune -f

Total reclaimed space: 0B


In [57]:
!docker image ls

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
img-ubuntu-sshd     latest              e543cd4d1c4e        25 minutes ago      208MB
ubuntu              16.04               c9d990395902        3 days ago          113MB


In [58]:
!docker rmi img-ubuntu-sshd 

Untagged: img-ubuntu-sshd:latest
Deleted: sha256:e543cd4d1c4e1d8e5391994b76a2dd5d89b7081ac8a7c2ad8a67648f34a2cca9
Deleted: sha256:c13b5f1a72dcf9d69d68a04bf9bb30edcfc9f790ce6c4da637eae8c68bab233d
Deleted: sha256:a6bde93d3bf0ff5850f2f633506248359724b7aced53ccf306cee85a888bbfd4
Deleted: sha256:74274971fc9df4f2830f14c8d35c2efecf727105d92f8fb86b639c7d5d6a0b77
Deleted: sha256:6be071bdafd2e826e2777a73e02e20367226ca7b42d5003ce08a02fd5b455896
Deleted: sha256:cfbd8b8cb452e26ba010718dc98fbacf8ea30a88f85c7e820461ce01d1ba6420
Deleted: sha256:55df085c65cfa522b52268b512e4037466951f10edb2805f5475bb159d6f4bcc
Deleted: sha256:eecb98f47155ed236a9d487b3157675d6e9ce48f342275b05c2f871022bf3275
Deleted: sha256:0f7bc431e42a24dc75f1a988d37d87ed24e756393cfda24b6332a8aaee3ad803
Deleted: sha256:64b02a6bc4126363600cf64fdc6fcf581a1b46953b38c8108a4339371a91691d
Deleted: sha256:4781826ad67550e19062d2c348e6e9d987ba98ab2a303c3a4f3ccefad4b3d316
Deleted: sha256:07ee6cbcecfa49fb52dee1e376a0cf6ef1dff1bd62d68215b095ac7ef836

In [36]:
!docker image prune --all -f

Deleted Images:
untagged: img-ubnt-sshd:latest
untagged: img-ubuntu-sshd:latest
deleted: sha256:b2ed71c5962e047e014a3c7fbe3b4438fda2b2b00c9fb0e05ccf92abde2636e8
deleted: sha256:c69c986782313d0bbbf314948ab2ad739d67943016e297eac750dca081811a6e
deleted: sha256:b8d79e5aea6d9ff3588b41a6595012463b35e7d8073c6f5d57029cb7f3c034d7
deleted: sha256:048534dfdc3f8ba5e45ac2a5546300a4603d7dd21a4a4d98f886fe40dfc94c89
deleted: sha256:a1efb26f56ad5ab5a60225cc6804b24254804c78b868b65fb163ce0ea947d137
deleted: sha256:2f0ca8f3e7600b5a23ac217019ae21eb0a9e62cb6cde412400b48b3dfc688ea7
deleted: sha256:dee5725ff26d8f0434267067f7ee13832490e6fd8c3a53b8262720ff5b9a92f4
deleted: sha256:5079295f9f125497bf205a18094711ea18caccf53b81714db1d7e22d3dabcdfb
deleted: sha256:6ba6afa4f92efbfc98396eaf9fbfee462412dfae85afe5d0292d607dcbe85005
deleted: sha256:f9de9f4c37b58c12b5cf1130f334eac5e8effa4c1e1522d3164f296877e9c19f
deleted: sha256:266154673edefd52f67fffa62b95a9308ce78f0f0ccf9d1bcd87d7d70089d309
deleted: sha256:b4bfb3d200184

In [39]:
!docker image ls

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE


### PS

In [59]:
!cd ./docker/img-ubuntu-sshd/ && docker build . -t img-ubuntu-sshd

Sending build context to Docker daemon   2.56kB
Step 1/10 : FROM ubuntu:16.04
 ---> c9d990395902
Step 2/10 : RUN apt-get update && apt-get install -y openssh-server iproute2
 ---> Running in 1b543f2f2a28
Get:1 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [102 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [102 kB]
Get:4 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [102 kB]
Get:5 http://archive.ubuntu.com/ubuntu xenial/universe Sources [9802 kB]
Get:6 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [78.5 kB]
Get:7 http://security.ubuntu.com/ubuntu xenial-security/main amd64 Packages [602 kB]
Get:8 http://security.ubuntu.com/ubuntu xenial-security/restricted amd64 Packages [12.7 kB]
Get:9 http://security.ubuntu.com/ubuntu xenial-security/universe amd64 Packages [431 kB]
Get:10 http://security.ubuntu.com/ubuntu xenial-security/multiverse amd64 Packages [34

Get:43 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 openssh-sftp-server amd64 1:7.2p2-4ubuntu2.4 [38.7 kB]
Get:44 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 openssh-server amd64 1:7.2p2-4ubuntu2.4 [335 kB]
Get:45 http://archive.ubuntu.com/ubuntu xenial/main amd64 python3-pkg-resources all 20.7.0-1 [79.0 kB]
Get:46 http://archive.ubuntu.com/ubuntu xenial/main amd64 python3-chardet all 2.3.0-2 [96.2 kB]
Get:47 http://archive.ubuntu.com/ubuntu xenial/main amd64 python3-six all 1.10.0-3 [11.0 kB]
Get:48 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 python3-urllib3 all 1.13.1-2ubuntu0.16.04.1 [57.3 kB]
Get:49 http://archive.ubuntu.com/ubuntu xenial/main amd64 python3-requests all 2.9.1-3 [55.4 kB]
Get:50 http://archive.ubuntu.com/ubuntu xenial/main amd64 tcpd amd64 7.6.q-25 [23.0 kB]
Get:51 http://archive.ubuntu.com/ubuntu xenial/main amd64 ssh-import-id all 5.5-0ubuntu1 [10.2 kB]
debconf: delaying package configuration, since apt-utils is not inst

Preparing to unpack .../libkrb5-3_1.13.2+dfsg-5ubuntu2_amd64.deb ...
Unpacking libkrb5-3:amd64 (1.13.2+dfsg-5ubuntu2) ...
Selecting previously unselected package libgssapi-krb5-2:amd64.
Preparing to unpack .../libgssapi-krb5-2_1.13.2+dfsg-5ubuntu2_amd64.deb ...
Unpacking libgssapi-krb5-2:amd64 (1.13.2+dfsg-5ubuntu2) ...
Selecting previously unselected package libxmuu1:amd64.
Preparing to unpack .../libxmuu1_2%3a1.1.2-2_amd64.deb ...
Unpacking libxmuu1:amd64 (2:1.1.2-2) ...
Selecting previously unselected package openssh-client.
Preparing to unpack .../openssh-client_1%3a7.2p2-4ubuntu2.4_amd64.deb ...
Unpacking openssh-client (1:7.2p2-4ubuntu2.4) ...
Selecting previously unselected package wget.
Preparing to unpack .../wget_1.17.1-1ubuntu1.3_amd64.deb ...
Unpacking wget (1.17.1-1ubuntu1.3) ...
Selecting previously unselected package xauth.
Preparing to unpack .../xauth_1%3a1.0.9-1ubuntu2_amd64.deb ...
Unpacking xauth (1:1.0.9-1ubuntu2) ...
Selecting previously unselected package ncurses

 ---> Running in 52f6656d3442
Removing intermediate container 52f6656d3442
 ---> dc0b8a6546e0
Step 9/10 : EXPOSE 22
 ---> Running in 16b94e9dbac9
Removing intermediate container 16b94e9dbac9
 ---> fdc0fdd5489b
Step 10/10 : CMD ["/usr/sbin/sshd", "-D"]
 ---> Running in 0bdc896accb9
Removing intermediate container 0bdc896accb9
 ---> cc447c8aaf70
Successfully built cc447c8aaf70
Successfully tagged img-ubuntu-sshd:latest


In [60]:
!docker run -d -p 22022:22 --name cont-ubuntu-sshd img-ubuntu-sshd 

2a9f60befdece80be07e6420a00a9eca7a2d572eba5f565d67b4ea874782118f
